In [ ]:
pip install google-generativeai


In [ ]:
from typing import List
import google.generativeai as genai
import numpy as np


In [ ]:
text = """
Retrieval-Augmented Generation (RAG) is a technique that combines information retrieval
with generative AI. Instead of relying only on what the model has seen during training,
RAG retrieves relevant documents or text chunks from an external knowledge base.
These retrieved chunks are then given as additional context to the generative model,
helping it provide more accurate and up-to-date answers.
"""

In [ ]:
def chunk_text(text: str, chunk_size: int) -> List[str]:
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

chunks = chunk_text(text, chunk_size=10)
print("Chunks:", chunks)

Chunks: ['Retrieval-Augmented Generation (RAG) is a technique that combines information retrieval', 'with generative AI. Instead of relying only on what the', 'model has seen during training, RAG retrieves relevant documents or', 'text chunks from an external knowledge base. These retrieved chunks', 'are then given as additional context to the generative model,', 'helping it provide more accurate and up-to-date answers.']


In [ ]:
genai.configure(api_key="")   #Create your Gemeni API key from Google Studio and paste here

In [ ]:
def get_embedding(text: str):
    result = genai.embed_content(
        model="models/text-embedding-004",
        content=text
    )
    return np.array(result['embedding'], dtype=np.float32)

embeddings = [get_embedding(chunk) for chunk in chunks]

In [ ]:
embeddings

[array([-7.57163391e-02, -7.70311477e-03, -3.29039879e-02, -1.26882910e-03,
        -2.54337564e-02,  4.61496692e-03,  4.54436839e-02, -2.83226371e-03,
         2.61141844e-02, -6.10436462e-02, -6.40140548e-02, -1.16799371e-02,
         3.68849328e-03,  7.10905939e-02, -4.55670319e-02, -3.61125395e-02,
         2.62700375e-02,  4.31533232e-02, -4.29086015e-02, -2.02519558e-02,
         3.12847011e-02,  7.71351717e-03, -4.43254746e-02, -8.76657516e-02,
         3.17645893e-02,  6.07618783e-03,  2.41329484e-02,  5.94826341e-02,
         3.00819776e-03, -2.74820384e-02,  2.36301254e-02,  5.95427193e-02,
         5.45834489e-02,  2.24621408e-03, -1.24983257e-02,  5.45723625e-02,
         1.77817196e-02, -1.53183332e-02,  1.68149900e-02, -3.77116315e-02,
        -3.13949324e-02,  7.26437336e-03, -5.32801356e-03,  1.47161761e-03,
        -1.60893402e-03, -3.25179510e-02,  4.57456559e-02,  9.82496589e-02,
        -4.49395329e-02,  5.32023609e-02, -4.19929773e-02, -7.53661543e-02,
        -1.8

In [ ]:
# Storing in the vector database

In [ ]:
pip install faiss-cpu

In [ ]:
import faiss

In [ ]:
dimension = len(embeddings[0])
print (dimension)
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))


768


In [1]:
#Semantic Search

In [ ]:
def semantic_search(query: str, top_k: int = 2):
    # Convert query into embedding
    q_emb = get_embedding(query).reshape(1, -1)

    # Search in FAISS index
    distances, indices = index.search(q_emb, top_k)

    # Just return the matching chunks
    results = [chunks[i] for i in indices[0]]
    return results

In [ ]:
#Asking a Query

In [ ]:
query = "What is RAG in AI?"
retrieved_context = " ".join(semantic_search(query, top_k=2))

prompt = f"Answer the question based on the context:\n\nContext: {retrieved_context}\n\nQuestion: {query}"

response = genai.GenerativeModel("gemini-2.5-flash").generate_content(prompt)

print("\nQuery:", query)
print("Retrieved Context:", retrieved_context)
print("RAG Answer:", response.text)


Query: What is RAG in AI?
Retrieved Context: Retrieval-Augmented Generation (RAG) is a technique that combines information retrieval model has seen during training, RAG retrieves relevant documents or
RAG Answer: Based on the context, Retrieval-Augmented Generation (RAG) is a technique that combines information retrieval and retrieves relevant documents.
